In [ ]:
import numpy as np
import qubex as qx
from qubex.analysis.fitting import fit_rotation
from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    SimulationResult,
    Transmon,
)

In [ ]:
# units: ns, GHz
transmon_dimension = 3
control_qubit_label = "Q0"
target_qubit_label = "Q1"
control_qubit_frequency = 7.648
target_qubit_frequency = 8.275
anharmonicity = -0.333
relaxation_rate = 5e-5
dephasing_rate = 5e-5
coupling_strength = 0.01

control_drive_frequency = control_qubit_frequency
target_drive_frequency = target_qubit_frequency

drag_duration = 16

cr_duration = 1000
cr_amplitude = 0.2
cr_ramptime = 50
cr_phase = 0.25 * np.pi

crosstalk_ratio = 0.01
crosstalk_amplitude = cr_amplitude * crosstalk_ratio
crosstalk_phase = 0.5 * np.pi

In [ ]:
control_qubit = Transmon(
    label=control_qubit_label,
    dimension=transmon_dimension,
    frequency=control_qubit_frequency,
    anharmonicity=anharmonicity,
    relaxation_rate=relaxation_rate,
    dephasing_rate=dephasing_rate,
)

target_qubit = Transmon(
    label=target_qubit_label,
    dimension=transmon_dimension,
    frequency=target_qubit_frequency,
    anharmonicity=anharmonicity,
    relaxation_rate=relaxation_rate,
    dephasing_rate=dephasing_rate,
)

couplings = [
    Coupling(
        pair=(control_qubit_label, target_qubit_label),
        strength=coupling_strength,
    ),
]

system = QuantumSystem(
    objects=[control_qubit, target_qubit],
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [ ]:
def drag_pi_pulse(
    duration: float,
    anharmonicity: float = anharmonicity,
) -> qx.Pulse:
    pulse = qx.pulse.Drag(
        duration=duration,
        amplitude=1,
        beta=-0.5 / (2 * np.pi * anharmonicity),
    )
    norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
    pulse = pulse.scaled(norm_factor)
    return pulse


pi_pulse = drag_pi_pulse(
    duration=drag_duration,
    anharmonicity=control_qubit.anharmonicity,
)
pi_pulse.plot()

result = simulator.mesolve(
    controls=[
        Control(
            target=control_qubit_label,
            frequency=control_drive_frequency,
            waveform=pi_pulse,
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere(control_qubit_label)
result.show_last_population(control_qubit_label)

In [ ]:
def cr_sequence(
    control_drive_frequency: float,
    target_drive_frequency: float,
    cr_amplitude: float,
    cr_duration: float,
    cr_ramptime: float,
    cr_phase: float,
    crosstalk_amplitude: float = 0.0,
    crosstalk_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    echo: bool = False,
):
    cr_waveform = qx.pulse.FlatTop(
        duration=cr_duration,
        amplitude=2 * np.pi * cr_amplitude,
        tau=cr_ramptime,
        phase=cr_phase,
    )
    cancel_waveform = qx.pulse.FlatTop(
        duration=cr_duration,
        amplitude=2 * np.pi * cancel_amplitude,
        tau=cr_ramptime,
        phase=cancel_phase,
    )
    crosstalk_waveform = qx.pulse.FlatTop(
        duration=cr_duration,
        amplitude=2 * np.pi * crosstalk_amplitude,
        tau=cr_ramptime,
        phase=crosstalk_phase,
    )
    with qx.PulseSchedule(
        [
            qx.PulseChannel(
                label="CR",
                frequency=target_drive_frequency,
                target=control_qubit_label,
            ),
            qx.PulseChannel(
                label="Crosstalk",
                frequency=target_drive_frequency,
                target=target_qubit_label,
            ),
            qx.PulseChannel(
                label="Target",
                frequency=target_drive_frequency,
                target=target_qubit_label,
            ),
        ],
    ) as cr:
        cr.add("CR", cr_waveform)
        cr.add("Target", cancel_waveform)
        cr.add("Crosstalk", crosstalk_waveform)

    if not echo:
        return cr
    else:
        with qx.PulseSchedule(
            [
                qx.PulseChannel(
                    label="Control",
                    frequency=control_drive_frequency,
                    target=control_qubit_label,
                ),
                qx.PulseChannel(
                    label="Target",
                    frequency=target_drive_frequency,
                    target=target_qubit_label,
                ),
                qx.PulseChannel(
                    label="CR",
                    frequency=target_drive_frequency,
                    target=control_qubit_label,
                ),
                qx.PulseChannel(
                    label="Crosstalk",
                    frequency=target_drive_frequency,
                    target=target_qubit_label,
                ),
            ],
        ) as ecr:
            ecr.call(cr)
            ecr.barrier()
            ecr.add("Control", pi_pulse)
            ecr.barrier()
            ecr.call(cr.scaled(-1))
            ecr.barrier()
            ecr.add("Control", pi_pulse)
        return ecr

In [ ]:
def simulate_cr(
    control_drive_frequency: float,
    target_drive_frequency: float,
    cr_amplitude: float,
    cr_duration: float,
    cr_ramptime: float,
    cr_phase: float,
    crosstalk_amplitude: float = 0.0,
    crosstalk_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    echo: bool = False,
    control_state: str = "0",
    n_samples: int = 256,
    plot: bool = False,
) -> SimulationResult:
    controls = cr_sequence(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration,
        cr_ramptime=cr_ramptime,
        cr_phase=cr_phase,
        crosstalk_amplitude=crosstalk_amplitude,
        crosstalk_phase=crosstalk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        echo=echo,
    )
    initial_state = system.state(
        {
            control_qubit.label: control_state,
            target_qubit.label: "0",
        },
    )
    result = simulator.mesolve(
        controls=controls,
        initial_state=initial_state,
        n_samples=n_samples,
    )
    if plot:
        result.plot_bloch_vectors(control_qubit_label)
        result.plot_bloch_vectors(target_qubit_label)
        result.display_bloch_sphere(control_qubit_label)
        result.display_bloch_sphere(target_qubit_label)
    return result

In [ ]:
result_0 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    control_state="0",
    plot=True,
)

In [ ]:
result_1 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    control_state="1",
    plot=True,
)

In [ ]:
def hamiltonian_tomography(
    control_drive_frequency: float,
    target_drive_frequency: float,
    cr_amplitude: float,
    cr_duration: float,
    cr_ramptime: float,
    cr_phase: float,
    cross_talk_amplitude: float = 0.0,
    cross_talk_phase: float = 0.0,
    cancel_amplitude: float = 0.0,
    cancel_phase: float = 0.0,
    n_samples: int = 100,
    plot: bool = False,
) -> dict:
    result_0 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration,
        cr_ramptime=cr_ramptime,
        cr_phase=cr_phase,
        crosstalk_amplitude=cross_talk_amplitude,
        crosstalk_phase=cross_talk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        control_state="0",
        n_samples=n_samples,
        plot=plot,
    )
    result_1 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration,
        cr_ramptime=cr_ramptime,
        cr_phase=cr_phase,
        crosstalk_amplitude=cross_talk_amplitude,
        crosstalk_phase=cross_talk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        control_state="1",
        n_samples=n_samples,
        plot=plot,
    )

    times = result_0.times
    vectors_0 = result_0.get_bloch_vectors(target_qubit.label)
    vectors_1 = result_1.get_bloch_vectors(target_qubit.label)

    indices = (times >= cr_ramptime) & (times < times[-1] - cr_ramptime)
    times_ = times[indices] - cr_ramptime * 0.5
    vectors_0_ = vectors_0[indices]
    vectors_1_ = vectors_1[indices]

    fit_0 = fit_rotation(
        times_,
        vectors_0_,
        plot=plot,
    )
    fit_1 = fit_rotation(
        times_,
        vectors_1_,
        plot=plot,
    )
    Omega_0 = fit_0["Omega"]
    Omega_1 = fit_1["Omega"]
    Omega = np.concatenate(
        [
            0.5 * (Omega_0 + Omega_1),
            0.5 * (Omega_0 - Omega_1),
        ]
    )
    coeffs = dict(
        zip(
            ["IX", "IY", "IZ", "ZX", "ZY", "ZZ"],
            Omega / (2 * np.pi),
        )
    )

    for key, value in coeffs.items():
        print(f"{key}: {value * 1e3:+.6f} MHz")

    cr_phase_est = -np.arctan2(coeffs["ZY"], coeffs["ZX"])

    cancel_pulse = -(coeffs["IX"] + 1j * coeffs["IY"])
    cancel_amplitude_est = np.abs(cancel_pulse)
    cancel_phase_est = np.angle(cancel_pulse)

    return {
        "Omega": Omega,
        "coeffs": coeffs,
        "cr_phase_est": cr_phase_est,
        "cancel_amplitude_est": cancel_amplitude_est,
        "cancel_phase_est": cancel_phase_est,
    }

In [ ]:
tomography_1 = hamiltonian_tomography(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase,
    cross_talk_amplitude=crosstalk_amplitude,
    cross_talk_phase=crosstalk_phase,
    cancel_amplitude=0.0,
    cancel_phase=0.0,
    plot=True,
)

tomography_1

In [ ]:
cr_phase_est = tomography_1["cr_phase_est"]
cr_phase_est

In [ ]:
tomography_2 = hamiltonian_tomography(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase + cr_phase_est,
    cross_talk_amplitude=crosstalk_amplitude,
    cross_talk_phase=crosstalk_phase,
    cancel_amplitude=0,
    cancel_phase=0,
    plot=True,
)

tomography_2

In [ ]:
cancel_amplitude_est = tomography_2["cancel_amplitude_est"]
cancel_phase_est = tomography_2["cancel_phase_est"]

cancel_amplitude_est, cancel_phase_est

In [ ]:
tomography_3 = hamiltonian_tomography(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase + cr_phase_est,
    cross_talk_amplitude=crosstalk_amplitude,
    cross_talk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=cancel_phase_est,
    plot=True,
)

tomography_3

In [ ]:
cr_sequence(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase + cr_phase_est,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=0.0,
    echo=True,
).plot(
    title="Echoed Cross Resonance Sequence",
    width=800,
    divide_by_two_pi=True,
)

In [ ]:
def infidelity(
    control_drive_frequency: float,
    target_drive_frequency: float,
    cr_amplitude: float,
    cr_duration: float,
    cr_ramp: float,
    cr_phase: float,
    cancel_amplitude: float,
    cancel_phase: float,
):
    result_0 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration,
        cr_ramptime=cr_ramp,
        cr_phase=cr_phase,
        crosstalk_amplitude=crosstalk_amplitude,
        crosstalk_phase=crosstalk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        echo=True,
        control_state="0",
        n_samples=2,
        plot=False,
    )
    result_1 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration,
        cr_ramptime=cr_ramp,
        cr_phase=cr_phase,
        crosstalk_amplitude=crosstalk_amplitude,
        crosstalk_phase=crosstalk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        echo=True,
        control_state="1",
        n_samples=2,
        plot=False,
    )

    state_0 = result_0.get_bloch_vectors(target_qubit.label)[-1]
    state_1 = result_1.get_bloch_vectors(target_qubit.label)[-1]
    target_0 = np.array([0, -1, 0])
    target_1 = np.array([0, 1, 0])
    fidelity_0 = 1 - 0.5 * np.sum((state_0 - target_0) ** 2)
    fidelity_1 = 1 - 0.5 * np.sum((state_1 - target_1) ** 2)
    return 1 - 0.5 * (fidelity_0 + fidelity_1)

In [ ]:
from qubex.analysis import plot_xy

duration_range = np.arange(100, 400, 20)

objective_values = np.array(
    [
        infidelity(
            control_drive_frequency=control_drive_frequency,
            target_drive_frequency=target_qubit_frequency,
            cr_amplitude=cr_amplitude,
            cr_duration=duration,
            cr_ramp=cr_ramptime,
            cr_phase=cr_phase + cr_phase_est,
            cancel_amplitude=cancel_amplitude_est,
            cancel_phase=cancel_phase_est,
        )
        for duration in duration_range
    ]
)

plot_xy(duration_range, objective_values)

In [ ]:
from scipy.interpolate import interp1d

interpolator = interp1d(
    duration_range,
    objective_values,
    kind="cubic",
)

duration_fine = np.linspace(100, 300, 1000)
objective_values_fine = interpolator(duration_fine)

plot_xy(duration_fine, objective_values_fine)

T = duration_fine[np.argmin(objective_values_fine)]
T

In [ ]:
cr_duration_est = round(T / 2) * 2
cr_duration_est

In [ ]:
infidelity(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_qubit_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration_est,
    cr_ramp=cr_ramptime,
    cr_phase=cr_phase + cr_phase_est,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=cancel_phase_est,
)

In [ ]:
result_ecr_0 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude,
    cr_duration=cr_duration_est,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase + cr_phase_est,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=cancel_phase_est,
    control_state="0",
    echo=True,
    plot=True,
)

In [ ]:
result_ecr_1 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_duration=cr_duration_est,
    cr_amplitude=cr_amplitude,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase + cr_phase_est,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_est,
    cancel_phase=cancel_phase_est,
    control_state="1",
    echo=True,
    plot=True,
)

In [ ]:
def objective_func(x: np.ndarray):
    (
        # cr_amplitude,
        cr_phase,
        cancel_amplitude,
        cancel_phase,
    ) = x
    result_0 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration_est,
        cr_ramptime=cr_ramptime,
        cr_phase=cr_phase,
        crosstalk_amplitude=crosstalk_amplitude,
        crosstalk_phase=crosstalk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        echo=True,
        control_state="0",
        n_samples=2,
        plot=False,
    )
    result_1 = simulate_cr(
        control_drive_frequency=control_drive_frequency,
        target_drive_frequency=target_drive_frequency,
        cr_amplitude=cr_amplitude,
        cr_duration=cr_duration_est,
        cr_ramptime=cr_ramptime,
        cr_phase=cr_phase,
        crosstalk_amplitude=crosstalk_amplitude,
        crosstalk_phase=crosstalk_phase,
        cancel_amplitude=cancel_amplitude,
        cancel_phase=cancel_phase,
        echo=True,
        control_state="1",
        n_samples=2,
        plot=False,
    )
    state_0 = result_0.get_bloch_vectors(target_qubit.label)[-1]
    state_1 = result_1.get_bloch_vectors(target_qubit.label)[-1]
    target_0 = np.array([0, -1, 0])
    target_1 = np.array([0, 1, 0])
    fidelity_0 = 1 - 0.5 * np.sum((state_0 - target_0) ** 2)
    fidelity_1 = 1 - 0.5 * np.sum((state_1 - target_1) ** 2)
    return 1 - 0.5 * (fidelity_0 + fidelity_1)

In [ ]:
import cma

initial_guess = [
    # cr_amplitude,
    cr_phase + cr_phase_est,
    cancel_amplitude_est,
    0.0,
]

es = cma.CMAEvolutionStrategy(
    initial_guess,
    1.0,
    {
        "seed": 42,
        "ftarget": 1e-6,
        "timeout": 120,
        "bounds": [
            [
                # -1.0,
                -np.pi,
                -1.0,
                -np.pi,
            ],
            [
                # 1.0,
                np.pi,
                1.0,
                np.pi,
            ],
        ],
        "CMA_stds": [
            # 0.01,
            0.01,
            0.01,
            0.01,
        ],
    },
)

es.optimize(objective_func)

es.result.xbest

In [ ]:
objective_func(es.result.xbest)

In [ ]:
cr_amplitude_opt = cr_amplitude
cr_phase_opt = es.result.xbest[0]
cancel_amplitude_opt = es.result.xbest[1]
cancel_phase_opt = es.result.xbest[2]

In [ ]:
es.result.xbest

In [ ]:
cr_phase + cr_phase_est, cancel_amplitude_est, 0.0

In [ ]:
result_0 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude_opt,
    cr_duration=cr_duration_est,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase_opt,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_opt,
    cancel_phase=cancel_phase_opt,
    control_state="0",
    echo=True,
    plot=True,
)

In [ ]:
result_1 = simulate_cr(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude_opt,
    cr_duration=cr_duration_est,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase_opt,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_opt,
    cancel_phase=cancel_phase_opt,
    control_state="1",
    echo=True,
    plot=True,
)

In [ ]:
cr_sequence(
    control_drive_frequency=control_drive_frequency,
    target_drive_frequency=target_drive_frequency,
    cr_amplitude=cr_amplitude_opt,
    cr_duration=cr_duration_est,
    cr_ramptime=cr_ramptime,
    cr_phase=cr_phase_opt,
    crosstalk_amplitude=crosstalk_amplitude,
    crosstalk_phase=crosstalk_phase,
    cancel_amplitude=cancel_amplitude_opt,
    cancel_phase=cancel_phase_opt,
    echo=True,
).plot(
    title="Echoed Cross Resonance Sequence",
    width=800,
    divide_by_two_pi=True,
)